# Cycling Potential Table

Data Extraction used to complement the paper data (regarding the distribution of cycling potential among modals and tiers)

In [1]:
%load_ext autoreload
%autoreload 2
import saopaulo.load_trips as sptr
import saopaulo.maps_aux as aux
import saopaulo.cycling_infrastructure as cinfra
import saopaulo.choropleth as spchoro

from shapely.geometry import LineString, Point
#import saopaulo.choropleth_folium as spchoro

import saopaulo.sp_grid as gr

from bikescience.intersect_ways import geometry_intersection_length

import geopandas as gpd
import json
import pandas as pd
import numpy as np
from ipywidgets import interact_manual, widgets, fixed
from IPython.core.display import display, HTML, clear_output
import folium

import warnings
import requests
import os
import fnmatch as fnm
warnings.simplefilter('ignore')

gr.SP_LAT = -23.63
gr.SP_LON = -46.55


/home/pedro/.local/lib/python3.8/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
od_trips = pd.read_csv('bases/complete_csv.csv')
od_trips['final_potential_v2'] = (od_trips['inclination_potential'] + od_trips['distance_potential']) / 2
# fix FE_VIA with wrong format (because of thousand separator)
od_trips.loc[od_trips['FE_VIA'] >= 10000, 'FE_VIA'] = od_trips.loc[od_trips['FE_VIA'] >= 10000, 'FE_VIA'] / 1000
od_trips.dropna(inplace = True)
od_trips = gpd.GeoDataFrame(od_trips, crs={'init': 'epsg:4326'},
                 geometry=[LineString(eval(r)) for r in od_trips['route']])

In [3]:
display(od_trips['final_potential_v2'])
display(od_trips['modal'])
display(od_trips['FE_VIA'])

0         0.428496
1         0.576615
2         0.049649
3         0.240028
4         0.335606
            ...   
175076    0.485744
175077    0.772107
175078    0.634039
175079    0.743115
175080    0.621556
Name: final_potential_v2, Length: 175081, dtype: float64

0         pedestrian
1         pedestrian
2             subway
3             subway
4             bus_sp
             ...    
175076    pedestrian
175077    pedestrian
175078    pedestrian
175079    pedestrian
175080    pedestrian
Name: modal, Length: 175081, dtype: object

0         22.132647
1         22.132647
2         18.882487
3         18.882487
4         18.882487
            ...    
175076    89.330849
175077    55.549985
175078    55.549985
175079    55.549985
175080    55.549985
Name: FE_VIA, Length: 175081, dtype: float64

In [4]:
def generate_potential_df(
    pedestrian, car, motorcycle, bus, subway, train,
    tier):
    
    trips = od_trips.loc[od_trips['final_potential_v2'] > tier * .25]
    trips = trips.loc[od_trips['final_potential_v2'] <= (tier + 1) * .25]
    
    if pedestrian:
        trips = trips.loc[trips['modal'] == 'pedestrian']
    if car:
        trips = trips.loc[(trips['modal'] == 'car') | 
                          (trips['modal'] == 'taxi') | 
                          (trips['modal'] == 'taxi_app')]
    if motorcycle:
        trips = trips.loc[trips['modal'] == 'motorcycle']
    if bus:
        trips = trips.loc[(trips['modal'] == 'bus_sp') | 
                          (trips['modal'] == 'bus_inter') | 
                          (trips['modal'] == 'bus_otherc') |
                          (trips['modal'] == 'bus_school') |
                          (trips['modal'] == 'bus_private')]
    if subway:
        trips = trips.loc[(trips['modal'] == 'subway') | 
                          (trips['modal'] == 'monorail')]
    if train:
        trips = trips.loc[trips['modal'] == 'train']
    return round(sum(trips['FE_VIA']))
    

In [7]:
import datetime
modals_name = ['pedestrian', 'car', 'motorcycle', 'bus', 'subway', 'train']

potential_matrix = [[0 for _ in range(4)] for _ in range(6)]
for tier in range(4):
    print('-----------------------------')
    print('Tier: ', tier)
    for modal in range(6):
        modals=[False, False, False, False, False, False]
        modals[modal] = True
        potentials = generate_potential_df(modals[0], modals[1], modals[2], modals[3], modals[4], modals[5], tier)
        potential_matrix[modal][tier] = potentials
        
        print('\t', modals_name[modal], ': ', potentials)
        
display(potential_matrix)

        

-----------------------------
Tier:  0
	 pedestrian :  134296
	 car :  4997886
	 motorcycle :  590995
	 bus :  7625063
	 subway :  1902695
	 train :  1623135
-----------------------------
Tier:  1
	 pedestrian :  1854836
	 car :  2492405
	 motorcycle :  189578
	 bus :  3519013
	 subway :  1049525
	 train :  473379
-----------------------------
Tier:  2
	 pedestrian :  7497336
	 car :  2863395
	 motorcycle :  202707
	 bus :  2646874
	 subway :  330829
	 train :  150897
-----------------------------
Tier:  3
	 pedestrian :  3857013
	 car :  1736685
	 motorcycle :  92559
	 bus :  1314955
	 subway :  150054
	 train :  59301


[[134296, 1854836, 7497336, 3857013],
 [4997886, 2492405, 2863395, 1736685],
 [590995, 189578, 202707, 92559],
 [7625063, 3519013, 2646874, 1314955],
 [1902695, 1049525, 330829, 150054],
 [1623135, 473379, 150897, 59301]]